In [48]:
import symusic
from symusic.core import TrackTickList, NoteTickList, NoteTick
from symusic.types import Track

import os
from pathlib import Path
from dotenv import load_dotenv

load_dotenv()


def extract_melody(t: Track) -> Track:
    melody = []
    z: NoteTick
    # notes: NoteTickList = t.notes.filter(lambda note: note.duration >= 240, inplace=True)
    notes: NoteTickList = t.notes
    notes.sort(key=lambda x: x.start, inplace=True)
    z: NoteTick
    concurrent = []
    curr = None
    while notes:
        note = notes.pop(0)
        concurrent.append(note)
        if notes:
            note2 = notes[0]
            if note2.start > note.start:
                # pop highest pitch
                max = concurrent.pop()
                while concurrent:
                    a = concurrent.pop()
                    if a.pitch > max.pitch:
                        max = a
                if curr is None or max.pitch > curr.pitch or max.start >= curr.end - 100:
                    melody.append(max)
                    curr = max

    new_track = symusic.Track()
    for note in melody:
        new_track.notes.append(note)
    return new_track

project_dir = Path("/Users/xingjianliu/repos/symbolic-music-generation")
data_dir = project_dir / "data"
single_track_dir = data_dir / 'single_track'
counter = 0
for file in os.listdir(single_track_dir):
    score = Score(single_track_dir / file)
    a: Track = score.tracks[0]
    score.tracks[0] = extract_melody(a)
    score.dump_midi("./out.mid")
    if counter == 2:
        break



